In [1]:
import numpy as np
import pandas as pd

In [2]:
from energy_consumption.models.ec_model1 import get_ec_forecasts_model1

energy_forecasts_m1 = get_ec_forecasts_model1()

if 'date_time' in energy_forecasts_m1.columns:
    energy_forecasts_m1 = energy_forecasts_m1.drop(columns=['date_time'])

100%|██████████| 100/100 [01:00<00:00,  1.65it/s]


                     winter  spring  autumn  working_day  saturday  hour_1  \
date_time                                                                    
2023-11-15 13:00:00       0       0       1            1         0       0   
2023-11-15 14:00:00       0       0       1            1         0       0   
2023-11-15 15:00:00       0       0       1            1         0       0   
2023-11-15 16:00:00       0       0       1            1         0       0   
2023-11-15 17:00:00       0       0       1            1         0       0   
...                     ...     ...     ...          ...       ...     ...   
2023-11-20 08:00:00       0       0       1            1         0       0   
2023-11-20 09:00:00       0       0       1            1         0       0   
2023-11-20 10:00:00       0       0       1            1         0       0   
2023-11-20 11:00:00       0       0       1            1         0       0   
2023-11-20 12:00:00       0       0       1            1        

In [3]:
energy_forecasts_m1

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-15,energy,36 hour,52.817001,58.868224,62.165334,64.109492,68.112249
1,2023-11-15,energy,40 hour,54.246501,59.096250,60.941734,61.107771,64.174999
2,2023-11-15,energy,44 hour,36.824750,35.245913,41.520599,47.511822,50.099999
3,2023-11-15,energy,60 hour,47.577251,48.787226,56.146085,61.662744,65.257500
4,2023-11-15,energy,64 hour,49.006751,49.015251,54.922485,58.661023,61.320250
5,2023-11-15,energy,68 hour,37.422001,36.129162,36.902597,42.004073,45.801999


In [4]:
from dax.models.dax_model1 import get_dax_forecasts_model1
df_sub_dax = get_dax_forecasts_model1()
df_sub_dax

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
2023-11-16 00:00:00+01:00,2023-11-15,DAX,1 day,-3.321265,-0.551383,0.060969,0.720873,2.719772
2023-11-17 00:00:00+01:00,2023-11-15,DAX,2 day,-4.087895,-0.823250,0.072355,0.987420,3.570492
2023-11-20 00:00:00+01:00,2023-11-15,DAX,5 day,-5.259161,-0.917655,0.161798,1.224285,4.628697
2023-11-21 00:00:00+01:00,2023-11-15,DAX,6 day,-6.282629,-1.044070,0.186070,1.437087,5.367396
2023-11-22 00:00:00+01:00,2023-11-15,DAX,7 day,-6.735742,-1.091646,0.326190,1.566328,6.259934


In [1]:
from dax.models.basicGARCH import get_dax_forecasts_basicGARCH
df_sub2_dax = get_dax_forecasts_basicGARCH()
df_sub2_dax

ImportError: cannot import name 'get_dax_forecasts_basicGARCH' from 'dax.models.basicGARCH' (C:\Users\Maria\Documents\Studium\Pyhton Projekte\PTSFC\dax\models\basicGARCH.py)

In [5]:
from datetime import datetime
import pandas as pd 

weeks = ['0 week', '1 week', '2 week', '3 week', '4 week']
date_st = (datetime.today().strftime('%Y-%m-%d'))

df_sub_inf = pd.DataFrame({
    "forecast_date": date_st,
    "target": "infections",
    "horizon": weeks,
    "q0.025": np.nan,
    "q0.25": np.nan,
    "q0.5": np.nan,
    "q0.75": np.nan,
    "q0.975": np.nan})

In [6]:

submission_frame = pd.merge(df_sub_dax, energy_forecasts_m1, how='outer')
submission_frame = pd.merge(submission_frame, df_sub_inf, how='outer')

In [8]:
submission_frame

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-15,DAX,1 day,-3.321265,-0.551383,0.060969,0.720873,2.719772
1,2023-11-15,DAX,2 day,-4.087895,-0.823250,0.072355,0.987420,3.570492
2,2023-11-15,DAX,5 day,-5.259161,-0.917655,0.161798,1.224285,4.628697
3,2023-11-15,DAX,6 day,-6.282629,-1.044070,0.186070,1.437087,5.367396
4,2023-11-15,DAX,7 day,-6.735742,-1.091646,0.326190,1.566328,6.259934
5,2023-11-15,energy,36 hour,52.817001,58.868224,62.165334,64.109492,68.112249
6,2023-11-15,energy,40 hour,54.246501,59.096250,60.941734,61.107771,64.174999
7,2023-11-15,energy,44 hour,36.824750,35.245913,41.520599,47.511822,50.099999
8,2023-11-15,energy,60 hour,47.577251,48.787226,56.146085,61.662744,65.257500
9,2023-11-15,energy,64 hour,49.006751,49.015251,54.922485,58.661023,61.320250


In [10]:
date_string = date_st
date_obj = datetime.strptime(date_string, "%Y-%m-%d")
formatted_date = date_obj.strftime("%Y%m%d")

submission_frame.to_csv(
    str('C:/Users/Maria/Documents/Studium/Pyhton Projekte/PTSFC/submission_files/' +
        formatted_date + '_BrienneVonTarth'),
    index=False, na_rep="NA")